In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import pandas as pd

from utils.db_util import create_postgres_engine
from utils.query_util import query_person_in_org, query_organization_stats

In [4]:
db_conn = create_postgres_engine()

In [5]:
# company_df = pd.read_csv('../../data/query_result/filtered_company_list_100_char_count_cutoff.csv')

# org_profiles = company_df['org_profile_link'].tolist()

# def escape_single_quote(text):
#     """Since postgres does not like single quote in text"""
#     text = text.replace("'", "''")
#     return text

# org_profiles = [escape_single_quote(t) for t in org_profiles]

# len(org_profiles)

In [6]:
%%time

org_stat_df = query_organization_stats(db_conn, 
                                       org_profile_links=None,
                                       min_word_length=100,
                                       min_person_count_per_org=1000)

CPU times: user 3.55 ms, sys: 12 ms, total: 15.6 ms
Wall time: 1min 53s


In [7]:
org_stat_df.shape

(695, 4)

In [8]:
org_stat_df.head(10)

,org_profile_link,count_person,avg_word_length,median_word_length
0,/company/us-army?trk=ppro_cprof,36620,187.371983,162
1,/company/ibm?trk=ppro_cprof,32292,191.326985,167
2,/company/united-states-air-force?trk=ppro_cprof,26261,188.169948,162
3,/company/us-navy?trk=ppro_cprof,26030,188.520246,162
4,/company/bank-of-america?trk=ppro_cprof,24173,182.656600,157
5,/company/hewlett-packard?trk=ppro_cprof,22949,189.057170,164
6,/company/at&t?trk=ppro_cprof,20455,186.869030,161
7,/company/jpmorgan-chase?trk=ppro_cprof,18804,180.191661,154
8,/company/microsoft?trk=ppro_cprof,16341,188.592191,164
9,/company/wells-fargo?trk=ppro_cprof,15945,182.006585,157


In [9]:
org_stat_df.to_csv('../../data/query_result/filtered_org_stat_100_word_count_cutoff.csv',
                   index=False)

### Get Aggregate Stat

In [23]:
reference_fp = '../../data/query_result/filtered_org_stat_100_char_count_cutoff.csv'

In [24]:
temp_df = pd.read_csv(reference_fp)

In [25]:
print('num companies: ', temp_df.shape[0])
print('avg employees: ', temp_df['count_person'].mean().round(2))
print('median employees: ', temp_df['count_person'].median().round(2))
print('avg word length: ', temp_df['avg_word_length'].mean().round(2))

num companies:  1995
avg employees:  3312.12
median employees:  1840.0
avg word length:  107.82


In [31]:
temp_df.sample(10)

,org_profile_link,count_person,avg_word_length,median_word_length
679,/company/atk?trk=ppro_cprof,2510,106.211155,82
1890,/company/meritor?trk=ppro_cprof,1045,102.529187,81
556,/company/apex-systems?trk=ppro_cprof,2970,124.245791,101
704,/company/providence-health-and-services?trk=pp...,2439,106.871259,82
1556,/company/carlson-marketing?trk=ppro_cprof,1255,118.082072,94
1979,/company/pvh?trk=ppro_cprof,1007,96.778550,75
1407,/company/kci-medical?trk=ppro_cprof,1379,103.762146,82
877,/company/bayer?trk=ppro_cprof,2057,110.960136,84
1762,/company/western-kentucky-university?trk=ppro_...,1120,98.932143,76
659,/company/blackrock?trk=ppro_cprof,2585,103.341586,79


### Comparison

In [ ]:
new_q_df = pd.read_csv('../../data/query_result/filtered_org_stat_100_char_count_cutoff.csv')
old_q_df = pd.read_csv('../../data/query_result/filtered_company_list_100_char_count_cutoff.csv')

In [ ]:
new_q_df.shape, old_q_df.shape

In [ ]:
new_q_df.head()

In [ ]:
old_q_df.head()

In [ ]:
new_q_df = new_q_df[['org_profile_link', 'count_person']].rename(
    columns={'count_person': 'count_person_new'}).copy()

In [ ]:
merged = old_q_df.merge(new_q_df, on='org_profile_link')

In [ ]:
merged.shape

In [ ]:
(merged['count_person'] - merged['count_person_new']).abs().max()